# Library/Comon

In [ ]:
try:
  ! pip install dill
  import dill
except:
  import dill


In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


import os



import warnings; warnings.filterwarnings(action='once')

from tqdm import tqdm

import datetime

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

import random
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
file = 'all_hour1800_300422.csv'
# all_ = pd.read_csv(DIR + 'Data/all/' + file, index_col=[0])
all_ = pd.read_csv(file, index_col=[0])

# all[['Date',	'Symbol', 'Open',	'High',	'Low',	'Close',	'Volume']].to_csv(DIR + 'Data/all/' + 'all_hour1800_300422.csv')


C:\Users\afecn\anaconda3.1\envs\ml\lib\site-packages\pandas\core\indexes\base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [ ]:
all = all_.iloc[-100000:, :].reset_index(drop = True)
all.drop(columns = ['Open', 'High', 'Volume'], inplace = True)

y_pers = np.where(all['Symbol'].shift(-1) == all['Symbol'].shift(0), (all['Close'].shift(-1)/all['Close'] - 1)*100, 0)
y = np.where(y_pers  > 0, 1, 0)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(all, y)
X_train = all[:int(len(all)*0.75)]
X_test = all[int(len(all)*0.75):]
X_test = X_test.reset_index(drop = True)

y_train = y[:int(len(all)*0.75)]
y_test = y[int(len(all)*0.75):]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(75000, 4) (75000,) (25000, 4) (25000,)


## Pipline

In [ ]:
class ExperimentalTransformer(BaseEstimator, TransformerMixin):

  def fit(self, X, y = None):

    return self

  def transform(self, X, y = None):

    X_ = X.copy() 
    print(X_)
    X_ = self.data(X_)
    X_ = self.use_lfl(X_)
    X_ = self.hide(X_)
    X_ = self.generate(X_, 9, ['Cl_min_day'])[['Cl_min_day_' + str(i) for i in range(9)]].fillna(0)

    return X_
  def generate(self, data_, range_, cols = ['Cl_min_day']):
      features = []
      for col in cols:
        empty = np.empty(shape = (data_.shape[0], range_))
        for sym in tqdm(data_['Symbol'].unique()):
          np_col = 0
          df_sym = data_[data_['Symbol'] == sym]  
          for i in (range(range_)):
            empty[df_sym.index[0]:df_sym.index[-1] + 1, np_col] = df_sym[col].shift(i)
            np_col +=1
        feature = [str(col) + '_' + str(i) for i in range(range_)]
        # features.append(feature)
        # data_[feature] = empty
        # data_ = data_.join(pd.DataFrame(empty, columns=feature))

      return data_.join(pd.DataFrame(empty, columns=feature))
  def hide(self, df):
      df['Cl_min_day']      = (df['Close']/df['Min_day']-1)*100

      df['Proft1000']       = (df['Cl_firs_tomor']/df['Close'].shift(0)-1)*100

      return df

  def data(self, all):
      all['Date'] = pd.to_datetime(all['Date'])
      all['DateTime'] = all['Date']
      all['DateTime'] = pd.to_datetime(all['DateTime'])
      all['Date'] = all['DateTime'].dt.date
      all['Date'] = pd.to_datetime(all['Date'])
      # all = all[(all['DateTime'].dt.time >= datetime.time(10,10))&(all['DateTime'].dt.time <= datetime.time(18,30))]
      # all = all[(all['DateTime'].dt.year >=2011)]
      return all  
  def use_lfl(self, df_f_all):
      df_all = pd.DataFrame()
      for i in df_f_all['Symbol'].unique():
        df = df_f_all[df_f_all['Symbol'] == i].copy()
        self.lfl(df)
        df_all = pd.concat([df_all, df])
        df_all.reset_index(inplace=True, drop = True)
      return df_all

  def lfl(self, df_f_all):
      data_np = np.array(df_f_all['Date'].dt.date)
      close_np = np.array(df_f_all['Close'])
      # high_np = np.array(df_f_all['High'])
      low_np = np.array(df_f_all['Low'])
      # print(data_np.size == close_np.size == high_np.size == low_np.size)
      k1 = []
      Cl_last_yeast = np.zeros(shape=data_np.size)
      Cl_last_today = np.zeros(shape=data_np.size)
      Cl_firs_today = np.zeros(shape=data_np.size)
      Cl_firs_tomor = np.zeros(shape=data_np.size)
      Min_day = np.zeros(shape=data_np.size)
      Max_day = np.zeros(shape=data_np.size)

      for i in range(data_np.size):
        if i == 0:
          Cl_last_yeast[i] = close_np[i]
        elif data_np[i] == data_np[i-1]:
          Cl_last_yeast[i] = Cl_last_yeast[i-1]
        else: 
          Cl_last_yeast[i] = close_np[i-1]

      df_f_all['Cl_last_yeast'] = Cl_last_yeast

      # #Short
      k1 = []

      for i in range(data_np.size-1, -1, -1):
        # print(i)
        if i == data_np.size - 1:
          Cl_last_today[i]  =  close_np[i]
          k = 1

        elif data_np[i] == data_np[i+1]:

          Cl_last_today[i]  =  Cl_last_today[i+1]
          k += 1

        else:

          Cl_last_today[i]  = close_np[i]
          k = 1
        k1.append(k)

      df_f_all['Short'] = k1[::-1]  
      df_f_all['Cl_last_today'] = Cl_last_today

      # Long
      k1 = []

      for i in range(data_np.size):
        if i == 0:
          Cl_firs_today[i]   =  close_np[i]
          k = 1

        elif data_np[i] == data_np[i-1]:

          Cl_firs_today[i]   =  Cl_firs_today[i-1]
          k += 1

        else:

          Cl_firs_today[i]  = close_np[i]
          k = 1
        k1.append(k)

      df_f_all['Long'] = k1
      df_f_all['Cl_firs_today'] = Cl_firs_today 

      # Clo_tomor

      for i in range(data_np.size-1, -1, -1):

        if i == data_np.size - 1:
          Cl_firs_tomor[i]  =  close_np[i]

        elif data_np[i] == data_np[i+1]:
          Cl_firs_tomor[i]  =  Cl_firs_tomor[i+1]

        else:
          Cl_firs_tomor[i]  = close_np[i+1]

      df_f_all['Cl_firs_tomor'] = Cl_firs_tomor


      # MinD

      for i in range(data_np.size):
        if i == 0:
          Min_day[i] = low_np[i]

        elif data_np[i] == data_np[i-1]:
          if low_np[i] < Min_day[i-1]:
            Min_day[i] = low_np[i]
          else:
            Min_day[i] = Min_day[i-1]

        else: 
          Min_day[i] = low_np[i]

        df_f_all['Min_day'] = Min_day

In [ ]:
# with input transformation
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('transform_data', ExperimentalTransformer()),    # this will trigger a call to __init__
                       ('linear_model', LinearRegression())
                      #  ('GB', GradientBoostingClassifier())
])

print("fit pipeline 2")
pipe2.fit(X_train, y_train)

preds2 = pipe2.predict(X_test)


create pipeline 2
fit pipeline 2
                      Date Symbol        Low      Close
0      2017-08-09 16:00:00   TATN  391.70000  392.50000
1      2017-08-09 17:00:00   TATN  388.30000  390.65000
2      2017-08-10 10:00:00   TATN  391.35000  393.10000
3      2017-08-10 11:00:00   TATN  390.75000  391.60000
4      2017-08-10 12:00:00   TATN  391.00000  391.90000
...                    ...    ...        ...        ...
74995  2016-11-14 13:00:00   VTBR    0.06858    0.06866
74996  2016-11-14 14:00:00   VTBR    0.06840    0.06935
74997  2016-11-14 15:00:00   VTBR    0.06871    0.06877
74998  2016-11-14 16:00:00   VTBR    0.06850    0.06852
74999  2016-11-14 17:00:00   VTBR    0.06831    0.06836

[75000 rows x 4 columns]


C:\Users\afecn\anaconda3.1\envs\ml\lib\site-packages\pandas\core\indexes\base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 89.27it/s]
C:\Users\afecn\anaconda3.1\envs\ml\lib\site-packages\pandas\core\internals\blocks.py:962: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprec

                      Date Symbol         Low       Close
0      2016-11-15 10:00:00   VTBR     0.06783     0.06808
1      2016-11-15 11:00:00   VTBR     0.06801     0.06823
2      2016-11-15 12:00:00   VTBR     0.06800     0.06819
3      2016-11-15 13:00:00   VTBR     0.06803     0.06846
4      2016-11-15 14:00:00   VTBR     0.06833     0.06835
...                    ...    ...         ...         ...
24995  2021-11-01 13:00:00   YNDX  5863.80000  5895.80000
24996  2021-11-01 14:00:00   YNDX  5882.00000  5894.60000
24997  2021-11-01 15:00:00   YNDX  5883.60000  5894.00000
24998  2021-11-01 16:00:00   YNDX  5767.00000  5841.00000
24999  2021-11-01 17:00:00   YNDX  5827.00000  5876.40000

[25000 rows x 4 columns]


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 124.98it/s]
C:\Users\afecn\anaconda3.1\envs\ml\lib\site-packages\pandas\core\internals\blocks.py:962: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new = convert_scalar(new_values, new)


In [ ]:
# with open("D:/logreg_pipeline3.dill", "wb") as f:
#     dill.dump(pipe2, f)

# Добавил запрос

In [ ]:
import requests
import urllib.request
import json 


ar = requests.post('http://127.0.0.1:8180/predict', json=X_test[:20].to_json())
ar

<Response [200]>

In [ ]:
json.loads(ar.json())[1]

[0.4699121037,
 0.470785284,
 0.4738658043,
 0.4709035918,
 0.4761687491,
 0.4786982541,
 0.4758647877,
 0.4711836916,
 0.4771656822,
 0.4771497252,
 0.473833361,
 0.4705191009,
 0.4739086419,
 0.4713295764,
 0.476213263,
 0.4751604167,
 0.4749543299,
 0.4734393916,
 0.4737872537,
 0.4712832186]